# TensorFlow - Final Exercise Solution

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
  
nltk.download('stopwords')

import string

# Yelp reviews dataset: sentiment and star rating

### The sentiment of the reviews is either positive or negative. The star rating is either 1, 2, 4 or 5.

In [ ]:
import pandas as pd
yelp_ratings = pd.read_csv("yelp_ratings.csv") #44530 samples

yelp_reviews = yelp_ratings["text"].values

# Sentiment is either positive or negative 
yelp_reviews_sentiment = yelp_ratings["sentiment"].values

# Star rating is either 1, 2, 4 or 5
yelp_reviews_stars = yelp_ratings["stars"].values

## Sample Yelp review

In [ ]:
sample_number = 0
print(yelp_reviews[sample_number]+"\n")

if yelp_reviews_sentiment[sample_number] == 1:
    print(f"Sentiment: positive, Stars: {yelp_reviews_stars[sample_number]}")
else:
    print(f"Sentiment: negative, Stars: {yelp_reviews_stars[sample_number]}")

# 1.) Preprocess the data.

Convert to lowercase, tokenize, and remove stopwords and punctuation.

In [ ]:
stpwrds = stopwords.words('english') + list(string.punctuation)

processed_text = []
for review in yelp_reviews:
    processed_text.append([word for word in word_tokenize(review.lower()) 
                           if word not in stpwrds])

## Train/Test split

In [ ]:
training_sentences = processed_text[0:20000]
testing_sentences = processed_text[20000:]


training_labels = yelp_reviews_sentiment[0:20000]
testing_labels = yelp_reviews_sentiment[20000:]

# star ratings are from 0 to 5 (6 classes)
training_stars = yelp_reviews_stars[0:20000]
testing_stars = yelp_reviews_stars[20000:]

## Sentences sequence and padding

In [ ]:
tokenizer = Tokenizer(num_words=10000, oov_token="<oov>")
tokenizer.fit_on_texts(training_sentences)

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=120, 
                                padding="post", truncating="post")

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=120, 
                                padding="post", truncating="post")

# 2.) Create an LSTM model to predict review sentiment.

In [ ]:
sentiment_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 16, input_length=120),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
sentiment_model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Train the model.

In [ ]:
history = sentiment_model.fit(training_padded, training_labels, epochs=5)

## Evaluate the model's performance on the test set.

In [ ]:
sentiment_test_accuracy = sentiment_model.evaluate(testing_padded, testing_labels, verbose=2)[1]

In [ ]:
print("The accuracy on the sentiment test set is: {0:.2f}".format(sentiment_test_accuracy))

# 3.)  Create an LSTM model to predict review star rating.

In [ ]:
stars_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 16, input_length=120),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'softmax')
])

In [ ]:
stars_model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Train the model.

In [ ]:
history = stars_model.fit(training_padded, training_stars, epochs=5)

## Evaluate the model's performance on the test set.

In [ ]:
stars_test_accuracy = stars_model.evaluate(testing_padded, testing_stars, verbose=2)[1]

In [ ]:
print("The accuracy on the stars test set is: {0:.2f}".format(stars_test_accuracy))